Adesso hai già:

un motore di ranking + spiegazioni (rank_and_explain(query, top_k)) definito nello Step D,

che prende una descrizione tipo:

"Cerco una guardia tiratrice con ottime percentuali da 3, buon FT%, forte rimbalzista offensivo e buon difensore."

e restituisce:

un DataFrame con i giocatori ordinati per similarità,

una colonna explanation con una spiegazione in linguaggio naturale.

👉 Con lo Step E vogliamo costruire una piccola interfaccia da Talent Scout, in cui:

l’utente scrive da tastiera la descrizione del giocatore ideale,

il sistema risponde con una shortlist top-N + spiegazioni stampate in modo leggibile.

In [1]:
import numpy as np
import pandas as pd
import re
from sklearn.metrics.pairwise import cosine_similarity
import os
import joblib
import pandas as pd

# === PATH ===
MODELS_DIR = "../data/models"
DATA_CLEAN_PATH = "../data/drafted_cleaned.csv"

VECTORIZER_PATH = os.path.join(MODELS_DIR, "tfidf_vectorizer.joblib")
MATRIX_PATH = os.path.join(MODELS_DIR, "tfidf_matrix.joblib")
META_PATH = os.path.join(MODELS_DIR, "index_metadata.csv")

# === Carica TF-IDF ===
vectorizer = joblib.load(VECTORIZER_PATH)
tfidf_matrix = joblib.load(MATRIX_PATH)
metadata_df = pd.read_csv(META_PATH)

print("Indice caricato.")
print("TF-IDF matrix shape:", tfidf_matrix.shape)
print("Metadata shape (prima del merge):", metadata_df.shape)

# === Carica dataset completo (con Status) ===
full_df = pd.read_csv(DATA_CLEAN_PATH)
print("Dataset stats shape:", full_df.shape)

# === AGGIUNGI STATUS a metadata_df ===
if "Status" not in metadata_df.columns:
    print("Aggiungo colonna Status al metadata...")
    # aggiungo status SOLO per i player che esistono nel metadata
status_map = full_df.set_index("Player")["Status"].to_dict()
metadata_df["Status"] = metadata_df["Player"].map(status_map)

print("Controllo lunghezze:", len(metadata_df), tfidf_matrix.shape[0])


print("Metadata shape (dopo merge):", metadata_df.shape)
print("Colonne metadata_df:", metadata_df.columns.tolist())
print("Valori Status:", metadata_df["Status"].value_counts(dropna=False))



# =========================================
# 1. Normalizzazione testo
# =========================================

def normalize_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r"\s+", " ", text).strip()
    return text

# =========================================
# 2. Skill patterns (NO RUOLI)
# =========================================

SKILL_PATTERNS = {
    "shooting_3": [
        r"\b3 ?punti\b",
        r"\bda 3\b",
        r"\btiro da 3\b",
        r"\btripla\b",
        r"\bpercentuali.*3\b",
        r"\b3p\b",
        r"\b3p%\b",
        r"\bottimi.*3 punti\b",
        r"\beccellenti.*3 punti\b",
    ],
    "ft": [
        r"\btiri liberi\b",
        r"\bft\b",
        r"\bft%\b",
        r"\bliberi\b"
    ],
    "reb": [
        r"\brimbalzi\b",
        r"\brimbalzista\b"
    ],
    "reb_off": [
        r"\brimbalzo offensivo\b",
        r"\brimbalzista offensivo\b"
    ],
    "playmaking": [
        r"\bassist\b",
        r"\bassist\b",
        r"\bpassatore\b",
        r"\bplaymaker\b",
        r"\bcreatore di gioco\b"
    ],
    "scorer": [
        r"\bscorer\b",
        r"\brealizzatore\b",
        r"\bpunti\b",
        r"\btanti punti\b"
    ]
}


# =========================================
# ACTIVE PATTERNS
# =========================================

ACTIVE_PATTERNS = [
    r"\bin attività\b",
    r"\bancora attivo\b",
    r"\bgiocatore attuale\b",
    r"\battualmente attivo\b",
    r"\battivo oggi\b",
    r"\boggi gioca\b",
    r"\bgioca ora\b",
    r"\bnba attuale\b",
]



SEMANTIC_EXPANSIONS = {
    "shooting_3": "ottimo tiratore da 3 punti tiro da 3 3P% buon tiratore da 3",
    "ft": "buon tiratore di tiri liberi FT% tiri liberi",
    "reb": "rimbalzi forte rimbalzista",
    "reb_off": "rimbalzo offensivo rimbalzista offensivo",
    "playmaking": "assist buon passatore playmaker",
    "defense": "difensore buon difensore stoppate recuperi",
    "scorer": "punti realizzatore"
}

SEMANTIC_WEIGHTS = {
    "shooting_3": 4,
    "ft": 3,
    "reb": 3,
    "reb_off": 4,
    "playmaking": 3,
    "defense": 3,
    "scorer": 3
}

# =========================================
# 3. Parsing intenti
# =========================================

# =========================================
# Parsing intenti (skill + active)
# =========================================

def parse_query_intents(raw_query: str):
    q = normalize_text(raw_query)
    detected_skills = set()
    active_flag = False

    # detect skills
    for skill, patterns in SKILL_PATTERNS.items():
        for p in patterns:
            if re.search(p, q):
                detected_skills.add(skill)
                break

    # detect active
    for p in ACTIVE_PATTERNS:
        if re.search(p, q):
            active_flag = True
            break

    return {
        "normalized_query": q,
        "skills": sorted(list(detected_skills)),
        "active_only": active_flag
    }

# =========================================
# 4. Query pesata
# =========================================

def build_weighted_query(raw_query: str, intents: dict) -> str:
    base = intents.get("normalized_query", normalize_text(raw_query))
    boosted = [base]

    for skill in intents.get("skills", []):
        exp = SEMANTIC_EXPANSIONS.get(skill, "")
        w = SEMANTIC_WEIGHTS.get(skill, 1)
        boosted.append((" " + exp) * w)

    return " ".join(boosted).strip()

# =========================================
# AGGIUNTA STATUS a metadata_df (senza cambiare righe)
# =========================================

# mappa Player → Status
status_map = full_df.set_index("Player")["Status"].to_dict()
metadata_df["Status"] = metadata_df["Player"].map(status_map)

# =========================================
# 5. Motore IR avanzato
# =========================================


def search_players_scout(query: str, top_k: int = 10):
    intents = parse_query_intents(query)
    weighted_query = build_weighted_query(query, intents)

    q_vec = vectorizer.transform([weighted_query])
    sim = cosine_similarity(q_vec, tfidf_matrix).ravel()

    # === FILTRO GIOCATORI ATTIVI ===
    if intents["active_only"]:
        active_mask = (metadata_df["Status"] == "Active").fillna(False).to_numpy()

        # deve avere la stessa shape della similarity
        if len(active_mask) != len(sim):
            raise ValueError(f"Mismatch size: status={len(active_mask)}, sim={len(sim)}")

        sim = sim * active_mask

    # nessun match
    if sim.max() == 0:
        empty = metadata_df.iloc[:0].copy()
        empty.attrs["intents"] = intents
        empty.attrs["weighted_query"] = weighted_query
        return empty

    idx = np.argsort(sim)[::-1][:top_k]
    results = metadata_df.iloc[idx].copy()
    results["similarity"] = sim[idx]
    results.attrs["intents"] = intents
    results.attrs["weighted_query"] = weighted_query

    return results


# =========================================
# 6. Formatter numerici
# =========================================

def format_percent_it(value, decimals=1):
    if value is None or pd.isna(value):
        return None
    v = float(value)
    if v <= 1:
        v *= 100
    return f"{v:.{decimals}f}".replace(".", ",") + "%"

def format_number_it(value, decimals=1):
    if value is None or pd.isna(value):
        return None
    return f"{float(value):.{decimals}f}".replace(".", ",")

# =========================================
# 7. Spiegazioni giocatore
# =========================================

def build_explanation(row, intents):
    """
    Spiegazione basata SOLO sulle colonne disponibili:
    FG%, 3P%, FT%, MP.1, PTS.1, TRB.1, AST.1
    """
    skills = intents.get("skills", [])
    name = row.get("Player", "Il giocatore")

    reasons = [f"{name} è stato selezionato perché:"]

    # --- Tiro da 3 (3P%) ---
    p3 = row.get("3P%")
    if "shooting_3" in skills and p3 is not None and not pd.isna(p3):
        reasons.append(f"- buon tiratore da tre ({format_percent_it(p3)}).")

    # --- Tiri liberi (FT%) ---
    # gestisco sia 'FT%' che eventuali varianti tipo 'FT% '
    ft = None
    for col in ["FT%", "FT% ", "FT%.1"]:
        if col in row.index and not pd.isna(row[col]):
            ft = row[col]
            break

    if "ft" in skills and ft is not None:
        reasons.append(f"- buon tiratore ai liberi ({format_percent_it(ft)}).")

    # --- Rimbalzi (TRB.1 = rimbalzi per partita) ---
    trb_pg = row.get("TRB.1")
    if ("reb" in skills or "reb_off" in skills) and trb_pg is not None and not pd.isna(trb_pg):
        trb_str = format_number_it(trb_pg)
        # se hai chiesto esplicitamente "rimbalzista offensivo" enfatizzo di più
        if "reb_off" in skills:
            reasons.append(f"- ottimo contributo a rimbalzo ({trb_str} rimbalzi a partita).")
        else:
            reasons.append(f"- buon rimbalzista ({trb_str} rimbalzi a partita).")

    # --- Assist (AST.1 = assist per partita) ---
    ast_pg = row.get("AST.1")
    if "playmaking" in skills and ast_pg is not None and not pd.isna(ast_pg):
        ast_str = format_number_it(ast_pg)
        reasons.append(f"- buon passatore ({ast_str} assist a partita).")

    # --- Scoring (PTS.1 = punti per partita) ---
    pts_pg = row.get("PTS.1")
    if "scorer" in skills and pts_pg is not None and not pd.isna(pts_pg):
        pts_str = format_number_it(pts_pg)
        reasons.append(f"- buon realizzatore ({pts_str} punti a partita).")

    # --- Difesa: non hai DWS / STL / BLK, mettiamo frase generica ---
    if "defense" in skills:
        reasons.append("- profilo difensivo interessante sulla base delle statistiche disponibili.")

    # Se non siamo riusciti a usare nessuna stat specifica
    if len(reasons) == 1:
        reasons.append("- il suo profilo testuale è molto simile alla descrizione richiesta.")

    return "\n".join(reasons)

# =========================================
# 8. Ranking + spiegazioni
# =========================================

def rank_and_explain(query, top_k=5):
    base = search_players_scout(query, top_k)
    intents = base.attrs["intents"]

    # Se esiste player_id su entrambi, usa quello.
    # Altrimenti fai merge sul nome del giocatore.
    if "player_id" in base.columns and "player_id" in full_df.columns:
        merged = base.merge(full_df, on="player_id", how="left")
    else:
        merged = base.merge(full_df, on="Player", how="left")

    merged["explanation"] = merged.apply(lambda r: build_explanation(r, intents), axis=1)

    return merged, intents


# =========================================
# 9. Stampa formattata
# =========================================

def print_player_result(row, i):
    # Prende nome, anno, pick in modo robusto
    name = row.get("Player", "N/A")

    # DraftYear può essere DraftYear, DraftYear_x o DraftYear_y
    if "DraftYear" in row.index:
        year = row["DraftYear"]
    elif "DraftYear_x" in row.index:
        year = row["DraftYear_x"]
    elif "DraftYear_y" in row.index:
        year = row["DraftYear_y"]
    else:
        year = "N/A"

    # Pick può essere Pick, Pick_x o Pick_y
    if "Pick" in row.index:
        pick = row["Pick"]
    elif "Pick_x" in row.index:
        pick = row["Pick_x"]
    elif "Pick_y" in row.index:
        pick = row["Pick_y"]
    else:
        pick = "N/A"

    header = f"#{i+1} — {name} (Draft {year}, pick {pick})"
    print(header)
    print("-" * len(header))

    explanation = row.get("explanation", "")
    print(explanation)
    print("\n")


# =========================================
# 10. Interfaccia CLI
# =========================================

def run_talent_scout_cli(top_k=5):
    print("=== SISTEMA INTELLIGENTE PER TALENT SCOUT NBA ===")
    print("Scrivi una descrizione del giocatore che cerchi.")
    print("Digita 'exit' per uscire.\n")

    while True:
        query = input("Descrivi il giocatore ideale: ").strip()
        if query.lower() in ("exit", "esci", "quit", "q"):
            print("Chiusura sistema 👋")
            break

        results, intents = rank_and_explain(query, top_k)
        print(query)
        print("\nSkill individuate:", intents["skills"], "\n")

        for i, (_, row) in enumerate(results.iterrows()):
            print_player_result(row, i)
        print("\n")


Indice caricato.
TF-IDF matrix shape: (8323, 22019)
Metadata shape (prima del merge): (8323, 7)
Dataset stats shape: (8323, 36)
Controllo lunghezze: 8323 8323
Metadata shape (dopo merge): (8323, 7)
Colonne metadata_df: ['player_id', 'Player', 'DraftYear', 'Pick', 'PickBand', 'Status', 'text_profile']
Valori Status: Status
Retired    7867
Active      456
Name: count, dtype: int64


In [2]:
print(metadata_df.columns)


Index(['player_id', 'Player', 'DraftYear', 'Pick', 'PickBand', 'Status',
       'text_profile'],
      dtype='object')


In [3]:
metadata_df.head()



,player_id,Player,DraftYear,Pick,PickBand,Status,text_profile
0,0,Clifton McNeely,1947,1,Top10,Retired,Giocatore: Clifton McNeely. Proveniente da Tex...
1,1,Glen Selbo,1947,2,Top10,Retired,Giocatore: Glen Selbo. Proveniente da Wisconsi...
2,2,Bulbs Ehlers,1947,3,Top10,Retired,Giocatore: Bulbs Ehlers. Proveniente da Purdue...
3,3,Walt Dropo,1947,4,Top10,Retired,Giocatore: Walt Dropo. Proveniente da UConn. S...
4,4,Dick Holub,1947,5,Top10,Retired,Giocatore: Dick Holub. Proveniente da Long Isl...


In [4]:
run_talent_scout_cli(top_k=5)

=== SISTEMA INTELLIGENTE PER TALENT SCOUT NBA ===
Scrivi una descrizione del giocatore che cerchi.
Digita 'exit' per uscire.

Cerco un tiratore con ottime percentuali da 3, buone percentuali ai tiri liberi,forte passatore e che sia ritirato

Skill individuate: ['ft', 'playmaking', 'shooting_3'] 

#1 — LaMelo Ball (Draft 2020, pick 3)
-------------------------------------
LaMelo Ball è stato selezionato perché:
- buon tiratore da tre (36,4%).
- buon tiratore ai liberi (83,6%).
- buon passatore (7,4 assist a partita).


#2 — Deron Williams (Draft 2005, pick 3)
----------------------------------------
Deron Williams è stato selezionato perché:
- buon tiratore da tre (35,7%).
- buon tiratore ai liberi (82,2%).
- buon passatore (8,1 assist a partita).


#3 — Damon Stoudamire (Draft 1995, pick 7)
------------------------------------------
Damon Stoudamire è stato selezionato perché:
- buon tiratore da tre (35,7%).
- buon tiratore ai liberi (83,3%).
- buon passatore (6,1 assist a partita).




Con questo Step E hai:

trasformato il tuo motore di ranking + spiegazioni in una vera interfaccia da Talent Scout, anche se testuale;

ora un direttore sportivo/scout può:

scrivere in linguaggio naturale il profilo del giocatore ideale,

vedere una shortlist top-N,

leggere una spiegazione in italiano del perché ciascun giocatore è stato proposto.